In [2]:
from utils import Policy, logging
# from forward import flexgen
from test import test_hf_gen

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

checkpoint = "facebook/opt-125m" # 125m 6.7b 13b 30b
# checkpoint = "Salesforce/codegen-350M-mono"
# checkpoint = 'bigscience/bloom-560m'

policy = Policy(
    gpu_batch_size=2, 
    num_gpu_batches=4, 
    weights_gpu_percent=0.0, 
    weights_cpu_percent=0.3, 
    cache_gpu_percent=0.0, 
    cache_cpu_percent=0.2, 
    act_gpu_percent=0.0, 
    act_cpu_percent=0.5, 
    overlap=True, 
    pin_weight=True,
)


In [3]:
# forward.py: rewrite layer forward function

import torch
import functools 
import contextlib

from minibatch import get_size_info, load_kth_batch_inputs, concat_outputs
from utils import get_module_from_name


def reset_forward(model, layer_name):        
    layer = get_module_from_name(model, layer_name) 

    if hasattr(layer, "_flexgen_old_forward"):
        layer.forward = layer._flexgen_old_forward
        delattr(layer, "_flexgen_old_forward")
        logger.debug(f'{layer_name} from flexgen to old.')

    if hasattr(layer, "_test_old_forward"):
        layer.forward = layer._test_old_forward
        delattr(layer, "_test_old_forward")
        logger.debug(f'{layer_name} from test to old.')

def to_test_forward(mpl, layer_name, call_layer_log):
    layer = get_module_from_name(mpl.model, layer_name) 
    compute_device = 'cpu' 
    layer._test_old_forward = old_forward = layer.forward 

    @functools.wraps(old_forward)
    def new_forward(*args, **kwargs):
        mpl.load_layer_weights(layer_name, compute_device) 

        call_layer_log.append(layer_name)  # 

        with torch.no_grad():
            output = old_forward(*args, **kwargs)

        mpl.offload_layer_weights(layer_name)
        return output

    layer.forward = new_forward
    logger.debug(f'{layer_name} to test forward') 

@contextlib.contextmanager
def test(mpl, call_layer_log):
    model = mpl.model
    layer_names = mpl.layer_names

    # test run to get layer calling order
    for layer_name in layer_names:
        to_test_forward(mpl, layer_name, call_layer_log)
    yield 
    for layer_name in layer_names:
        reset_forward(model, layer_name)



/home/fsuser/miniconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-10-11 10:07:54,036 [instantiator.py:21 in <module>] INFO - Created a temporary directory at /tmp/tmp8tuytnca
2023-10-11 10:07:54,038 [instantiator.py:76 in _write] INFO - Writing /tmp/tmp8tuytnca/_remote_module_non_scriptable.py


In [35]:
ts = torch.tensor([[[1],[2],[3]]])

tensor([[[2]]])

In [45]:
from typing import Mapping, Tuple
import numpy as np 
import os 
from math import floor

class MixTensor:
    def __init__(
        self, 
        mix_data: Tuple, 
        split_dim: int, 
        device: torch.device, 
        shape: torch.Size,
        percents: Mapping[str, float],
        file_path: str
    ):
        self.mix_data = mix_data
        self.split_dim = split_dim 
        self.device = device 
        self.shape = shape 
        self.percents = percents
        self.file_path = file_path
    
    def size(self):
        return self.shape 
    
    @staticmethod
    def get_split_dim(tensor):
        dim_sizes = tensor.size()
        max_dim, max_size = -1, -1
        for dim, size in enumerate(dim_sizes):
            if size > max_size:
                max_size = size
                max_dim = dim 
        return max_dim 
    
    @staticmethod
    def tensor_dim_slice(tensor, dim, dim_slice):
        return tensor[(dim if dim >= 0 else dim + tensor.dim()) * (slice(None), ) + (dim_slice, )]
    
    @staticmethod
    def split_tensor(tensor, dim, percents):
        dim_size = tensor.size(dim)
        g_per, c_per, _ = [percents[dev] for dev in ['cuda', 'cpu', 'disk']]
        
        g_cut = floor(dim_size * g_per)
        c_cut = floor(dim_size * (g_per + c_per))

        g_data = MixTensor.tensor_dim_slice(tensor, dim, slice(0, g_cut))
        c_data = MixTensor.tensor_dim_slice(tensor, dim, slice(g_cut, c_cut))
        d_data = MixTensor.tensor_dim_slice(tensor, dim, slice(c_cut, dim_size))
        return g_data, c_data, d_data 

    @classmethod
    def from_tensor(
        cls, 
        tensor: torch.Tensor, 
        percents: Mapping[str, float],
        file_path: str 
    ):
        split_dim = cls.get_split_dim(tensor) 
        device = tensor.device 
        shape = tensor.shape
        
        g_data, c_data, d_data = cls.split_tensor(tensor, split_dim, percents) 
        
        g_data = g_data.to('cuda' if torch.cuda.is_available() else 'cpu') if g_data.numel() else None
        c_data = c_data.to('cpu') if c_data.numel() else None
        if d_data.numel():
            d_data = d_data.cpu().numpy()
            shape = d_data.shape
            dtype = d_data.dtype 

            fp = np.memmap(file_path, mode="w+", shape=shape, dtype=dtype)
            fp[:] = d_data[:]
            d_data = (shape, dtype)
        else:
            d_data = None 
        mix_data = (g_data, c_data, d_data)

        return cls(
            mix_data=mix_data,
            split_dim=split_dim,
            device=device,
            shape=shape,
            percents=percents,
            file_path=file_path
        )

    def to_tensor(self):
        g_data, c_data, d_data = self.mix_data 
        compute_device = self.device 

        tensor = []
        if g_data is not None:
            if g_data.device != torch.device(compute_device):
                g_data = g_data.to(compute_device) 
            tensor.append(g_data)
        if c_data is not None:
            if c_data.device != torch.device(compute_device):
                c_data = c_data.to(compute_device) 
            tensor.append(c_data)
        if d_data is not None:
            (shape, dtype) = d_data 
            d_data = np.memmap(self.file_path, shape=shape, dtype=dtype, mode='r')
            d_data = torch.from_numpy(d_data).to(compute_device)
            tensor.append(d_data)
            
        tensor = torch.cat(tensor, dim=self.split_dim) 
        return tensor 
            

    def __add__(self, mix_tensor):
        assert self.shape == mix_tensor.shape and type(self) == type(mix_tensor) # is same shape mix tensor
        res = self.to_tensor() + mix_tensor.to_tensor() 
        return self.from_tensor(res, self.percents, self.file_path)


x = torch.tensor([1,2,3])
m = MixTensor.from_tensor(x, percents={'cuda':0, 'cpu':0.5, 'disk':0.5}, file_path='test/m.dat')
m2 = MixTensor.from_tensor(x, percents={'cuda':0, 'cpu':0.5, 'disk':0.5}, file_path='test/m2.dat')
(m+m2).to_tensor()


[tensor([1]), tensor([2, 3])]
[tensor([1]), tensor([2, 3])]
[tensor([2]), tensor([4, 6])]


tensor([2, 4, 6])

In [ ]:



def to_flexgen_forward(mpl, j, compute_device):
    # rewrite the j-th layer's forward
    layer_name = mpl.layer_names[j]
    next_layer_name = mpl.layer_names[(j + 1) % len(mpl.layer_names)]

    ngb = mpl.policy.num_gpu_batches

    layer = get_module_from_name(mpl.model, layer_name)  
    if hasattr(layer, "_flexgen_old_forward"): return  
    
    layer._flexgen_old_forward = old_forward = layer.forward 

    @functools.wraps(old_forward)
    def new_forward(*args, **kwargs):
        # pre fwd: load curr & next weights, TODO: cuda stream
        mpl.load_layer_weights(layer_name, compute_device) 
        mpl.load_layer_weights(next_layer_name, compute_device) 
        
        # loop forward pass of K minibatches, TODO: cuda stream
        with torch.no_grad():
            logger.debug(f'args: {get_size_info(args)}')
            logger.debug(f'kwargs: {get_size_info(kwargs)}')
            
            outputs = []
            for k in range(ngb):
                logger.debug(f'layer: {layer_name}, batch: {k}')

                # 'pre' fwd: load curr & next inputs (activations, KV cache), store & offload prev 
                args_k = load_kth_batch_inputs(args, k, ngb)
                kwargs_k = load_kth_batch_inputs(kwargs, k, ngb)

                # TODO: load args, kwargs to compute device

                # the k-th fwd pass
                output = old_forward(*args_k, **kwargs_k)

                # TODO: 1) output: to mix, 2) args_k, kwargs_k: free


                outputs.append(output) 

            output = concat_outputs(outputs)
            logger.debug(f'outputs after concat: {get_size_info(output)}')                

        # post fwd: free curr weights
        mpl.offload_layer_weights(layer_name)
        return output

    layer.forward = new_forward
    logger.debug(f'{layer_name} to flexgen forward')

@contextlib.contextmanager 
def flexgen(checkpoint, policy):
    # init model 
    from model import ModelPolicyLoader
    mpl = ModelPolicyLoader(checkpoint, policy)
    mpl.init_all_weights() # init 

    # test run, get layer order
    call_layer_log = []
    with test(mpl, call_layer_log):
        from test import test_hf_gen
        test_hf_gen(mpl.checkpoint, mpl.model, 1,1, prompts=['0'])

    assert len(call_layer_log) == len(mpl.layer_names) and set(call_layer_log) == set(mpl.layer_names)
    mpl.layer_names = call_layer_log

    # rewrite layer forward
    for j, _ in enumerate(mpl.layer_names):
        compute_device = 'cpu'
        to_flexgen_forward(mpl, j, compute_device)
    yield mpl.model 
    for layer_name in mpl.layer_names:
        reset_forward(mpl.model, layer_name)
        

In [ ]:
with flexgen(checkpoint, policy) as model:
    num_prompts = policy.gpu_batch_size * policy.num_gpu_batches
    test_hf_gen(checkpoint, model, num_prompts)
